In [64]:
from typing import Dict,Text
import pandas as pd
import tensorflow as tf
import tensorflow_recommenders as tfrs
import numpy as np
import duckdb
con = duckdb.connect('database.db')

#PREPARE DATA

podcast_ratings_query = con.sql("select author_id as user_id, podcasts.title as podcast_title,rating from reviews "+ 
        "join categories using (podcast_id) " +
        "join podcasts using (podcast_id) where average_rating >= 0").to_df()

podcast_titles_query = con.sql("select podcasts.title as podcast_title from reviews "+ 
        "join categories using (podcast_id) " +
        "join podcasts using (podcast_id) where average_rating >= 0").to_df()

podcast_ratings = podcast_ratings_query.to_dict(orient='records')
podcast_titles = podcast_titles_query.to_dict(orient='records')
podcast_ratings_tf = tf.data.Dataset.from_tensor_slices(pd.DataFrame.from_dict(podcast_ratings).to_dict(orient="list"))
podcast_titles_tf = tf.data.Dataset.from_tensor_slices(pd.DataFrame.from_dict(podcast_titles).to_dict(orient="list"))

ratings_tf = podcast_ratings_tf.map(lambda x: {
    "podcast_title": x["podcast_title"],
    "user_id": x["user_id"],
    "user_rating": x["rating"]
})
titles_tf = podcast_titles_tf.map(lambda x: x["podcast_title"])

tf.random.set_seed(42)
shuffled = ratings_tf.shuffle(100_000, seed=42, reshuffle_each_iteration=False)

train_len = int(len(shuffled) * (2/3))
test_len = int(len(shuffled) - train_len)

train = shuffled.take(train_len)
test = shuffled.skip(train_len).take(test_len)

podcast_titles = titles_tf.batch(1_000)
user_ids = ratings_tf.batch(1_000_000).map(lambda x: x["user_id"])

unique_podcast_titles = np.unique(np.concatenate(list(podcast_titles)))
unique_user_ids = np.unique(np.concatenate(list(user_ids)))

In [65]:
#TRAIN MODEL

class PodcastlensModel(tfrs.Model):

  def __init__(self):
    super().__init__()
    embedding_dimension = 32
    self.user_model = tf.keras.Sequential([
      tf.keras.layers.StringLookup(
          vocabulary=unique_user_ids, mask_token=None),
      # We add an additional embedding to account for unknown tokens.
      tf.keras.layers.Embedding(len(unique_user_ids) + 1, embedding_dimension)
    ])

    self.podcast_model = tf.keras.Sequential([
      tf.keras.layers.StringLookup(
          vocabulary=unique_podcast_titles, mask_token=None),
      tf.keras.layers.Embedding(len(unique_podcast_titles) + 1, embedding_dimension)
    ])
    metrics = tfrs.metrics.FactorizedTopK(
      candidates=titles_tf.batch(128).map(self.podcast_model)
    )

    self.task = tfrs.tasks.Retrieval(
      metrics=metrics
    )

  def compute_loss(self, features: Dict[Text, tf.Tensor], training=False) -> tf.Tensor:
    # We pick out the user features and pass them into the user model.
    user_embeddings = self.user_model(features["user_id"])
    # And pick out the podcast features and pass them into the podcast model,
    # getting embeddings back.
    positive_podcast_embeddings = self.podcast_model(features["podcast_title"])

    # The task computes the loss and the metrics.
    return self.task(user_embeddings, positive_podcast_embeddings)

model = PodcastlensModel()
model.compile(optimizer=tf.keras.optimizers.Adamax(learning_rate=0.1))
cached_train = train.shuffle(100_000).batch(250).cache()
cached_test = test.batch(100).cache()
model.fit(cached_train, epochs=25)

# Create a model that takes in raw query features, and
index = tfrs.layers.factorized_top_k.Streaming(model.user_model,k=25)
# recommends podcasts out of the entire podcasts dataset.
index.index_from_dataset(
  tf.data.Dataset.zip((titles_tf.batch(100), titles_tf.batch(100).map(model.podcast_model)))
)

Epoch 1/25
3/3 [==============================] - 2s 98ms/step - factorized_top_k/top_1_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_5_categorical_accuracy: 0.0019 - factorized_top_k/top_10_categorical_accuracy: 0.0097 - factorized_top_k/top_50_categorical_accuracy: 0.0351 - factorized_top_k/top_100_categorical_accuracy: 0.0936 - loss: 700.5382 - regularization_loss: 0.0000e+00 - total_loss: 700.5382 
Epoch 2/25
3/3 [==============================] - 0s 88ms/step - factorized_top_k/top_1_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_5_categorical_accuracy: 0.0175 - factorized_top_k/top_10_categorical_accuracy: 0.0682 - factorized_top_k/top_50_categorical_accuracy: 0.2417 - factorized_top_k/top_100_categorical_accuracy: 0.4639 - loss: 615.8552 - regularization_loss: 0.0000e+00 - total_loss: 615.8552
Epoch 3/25
3/3 [==============================] - 0s 93ms/step - factorized_top_k/top_1_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_5_categorical_accuracy: 0

In [66]:
#PULL METRICS FOR MODEL ON TEST DATA 
model.evaluate(cached_test, return_dict=True)

3/3 [==============================] - 1s 97ms/step - factorized_top_k/top_1_categorical_accuracy: 0.5019 - factorized_top_k/top_5_categorical_accuracy: 0.5486 - factorized_top_k/top_10_categorical_accuracy: 0.5642 - factorized_top_k/top_50_categorical_accuracy: 0.6031 - factorized_top_k/top_100_categorical_accuracy: 0.6887 - loss: 214.4407 - regularization_loss: 0.0000e+00 - total_loss: 214.4407


{'factorized_top_k/top_1_categorical_accuracy': 0.5019454956054688,
 'factorized_top_k/top_5_categorical_accuracy': 0.548638105392456,
 'factorized_top_k/top_10_categorical_accuracy': 0.5642023086547852,
 'factorized_top_k/top_50_categorical_accuracy': 0.6031128168106079,
 'factorized_top_k/top_100_categorical_accuracy': 0.688715934753418,
 'loss': 134.0203857421875,
 'regularization_loss': 0,
 'total_loss': 134.0203857421875}

In [67]:
#PULL METRICS FOR MODEL ON TRAIN DATA 
model.evaluate(cached_train, return_dict=True)

3/3 [==============================] - 0s 84ms/step - factorized_top_k/top_1_categorical_accuracy: 0.5731 - factorized_top_k/top_5_categorical_accuracy: 0.6511 - factorized_top_k/top_10_categorical_accuracy: 0.6725 - factorized_top_k/top_50_categorical_accuracy: 0.7193 - factorized_top_k/top_100_categorical_accuracy: 0.8051 - loss: 405.7846 - regularization_loss: 0.0000e+00 - total_loss: 405.7846


{'factorized_top_k/top_1_categorical_accuracy': 0.5730994343757629,
 'factorized_top_k/top_5_categorical_accuracy': 0.6510721445083618,
 'factorized_top_k/top_10_categorical_accuracy': 0.6725146174430847,
 'factorized_top_k/top_50_categorical_accuracy': 0.719298243522644,
 'factorized_top_k/top_100_categorical_accuracy': 0.8050682544708252,
 'loss': 15.008101463317871,
 'regularization_loss': 0,
 'total_loss': 15.008101463317871}

In [68]:
#RUN PREDICTIONS ON TEST DATA 
num = 0 
threshold = 1
for element in test:
    user_id = element["user_id"].numpy().decode()
    _, titles = index(tf.constant([user_id]))
    unique_preds = np.unique(titles.numpy())
    unique_preds = [el.decode('UTF-8') for el in unique_preds]
    if len(unique_preds) > threshold:
        print("Top {} recommendations for user {}: {}".format(len(unique_preds),user_id,unique_preds))
        print("")
        num += 1
print("Number of users with more than {} recommendation: {}".format(threshold,num))

5 recommendations for user 2a54e87ac1ea627: ['Daily Solutions Podcast', 'Hear It Now (retired)', 'In Real Life with Emily and Kimzilla | WFMU', 'Sketched Out', 'The Fertility Warriors Podcast: Helping women survive infertility and trying to conceive']

5 recommendations for user 3f8437d8d907072: ['Daily Solutions Podcast', 'F*ck Like a Woman', 'Hear It Now (retired)', 'Maximiza Tu Negocio en Redes de Mercadeo', 'Rebel Chums']

2 recommendations for user 27d85c140f6e9ba: ['Last Week at the Movies Podcast', 'NFL no ProFootballcast com Antony Curti e Eduardo Miceli']

4 recommendations for user d732db86069c70c: ['Hear It Now (retired)', 'Maximiza Tu Negocio en Redes de Mercadeo', 'The Fertility Warriors Podcast: Helping women survive infertility and trying to conceive', 'Understanding Human Behavior - Video']

4 recommendations for user a52ebc918bc19d1: ['All of the Above radio', 'Hear It Now (retired)', 'Inglestotal : Cursos y clases gratis de Ingles', 'The Oatley Academy ArtCast']

4 re